## News :D



In [42]:
%load_ext autoreload
%autoreload 2

import requests
import re
import itertools
from bs4 import BeautifulSoup

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Installs:
lxmn
bs4

In [53]:
months = ['January', 'February','March','April','May','June','July','August','September','October','November','December']


# CNN
grouped_cnn_links = [[f'https://www.cnn.com/article/sitemap-{i}-{j}.html' for j in range(1,13)] for i in range(2014, 2025)]
cnn_links = list(itertools.chain.from_iterable(grouped_cnn_links))
cnn_dict = dict([(i+2014, list) for i, list in enumerate(grouped_cnn_links)])



# New York Times
grouped_nyt_links = [[f'https://www.nytimes.com/sitemap/{i}/{str(j).zfill(2)}/' for j in range(1,13)] for i in range(2014, 2025)]
nyt_links = list(itertools.chain.from_iterable(grouped_nyt_links))
nyt_dict = dict([(i+2014, list) for i, list in enumerate(grouped_nyt_links)])



# Business Insider
grouped_bi_links = [[f'https://www.businessinsider.com/sitemap/html/{i}-{j}.html' for j in range(1,13)] for i in range(2014, 2025)]
bi_links = list(itertools.chain.from_iterable(grouped_bi_links))
bi_dict = dict([(i+2014, list) for i, list in enumerate(grouped_bi_links)])



# The Hill
grouped_th_links = [[f'https://thehill.com/sitemap/{i}/{j}' for j in months] for i in range(2014, 2025)]
th_links = list(itertools.chain.from_iterable(grouped_th_links))
th_dict = dict([(i+2014, list) for i, list in enumerate(grouped_th_links)])



# New York Post
grouped_nyp_links = [[f'https://nypost.com/{i}/{str(j).zfill(2)}/' for j in range(1,13)] for i in range(2014, 2025)]
nyp_links = list(itertools.chain.from_iterable(grouped_nyp_links))
nyp_dict = dict([(i+2014, list) for i, list in enumerate(grouped_nyp_links)])



# Daily Mail
grouped_dm_links = [[f'https://www.dailymail.co.uk/home/sitemaparchive/month_{i}{str(j).zfill(2)}/.html' for j in range(1,13)] for i in range(2014, 2025)]
dm_links = list(itertools.chain.from_iterable(grouped_dm_links))
dm_dict = dict([(i+2014, list) for i, list in enumerate(grouped_dm_links)])



# Fox
fox_links = ['https://www.foxnews.com/sitemap.xml?type=articles'] + [f'https://www.foxnews.com/sitemap.xml?type=articles&page={i}' for i in range(2, 161)]